In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
from pmdarima.datasets import load_ausbeer

# dados trimestrais
series = load_ausbeer(as_series=True)[:-1]

In [ ]:
?load_ausbeer

In [ ]:
series.head()

In [ ]:
series.tail()

In [ ]:
series.plot()

In [ ]:
series.hist()

In [ ]:
from pmdarima.arima import ndiffs, nsdiffs


# kpss test
ndiffs(series, test='kpss')

In [ ]:
# augmented dickey-fuller
ndiffs(series, test='adf')

In [ ]:
# testes para sazonalidade
nsdiffs(series, test='ocsb', m=4)

In [ ]:
nsdiffs(series, test='ch', m=4)

In [ ]:
?series.diff

In [ ]:
# removendo a tendencia
series.diff(periods=1).plot()

In [ ]:
# removendo a sazonalidade
series.diff(periods=4).plot()

In [ ]:
# residuos
series.diff(periods=1).diff(periods=4).plot()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(series,
                            model='additive',
                            period=4)

plt = result.plot()
?seasonal_decompose

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt = plot_acf(series, lags=12)

In [ ]:
plt = plot_pacf(series, lags=12)

In [ ]:
plt = plot_acf(series.diff(periods=4)[4:], lags=12)

In [ ]:
plt = plot_acf(series.diff().diff(periods=4)[5:], lags=12)

In [ ]:
import os
os.listdir('..')

In [ ]:
# heterocedastidade
from src.heteroskedasticity import het_tests

p_value = het_tests(series, test='white')
# p_value = het_tests(series, test='breuschpagan')

p_value

In [ ]:
from scipy.stats import boxcox
from scipy.special import inv_boxcox

?boxcox
# box-cox
transformed_data, best_lambda = boxcox(series)

print(transformed_data[:5])
print(series.values[:5])
print(best_lambda)

In [ ]:
pd.Series(transformed_data).plot()

In [ ]:
series.plot()

In [ ]:
# revertendo
original_series = inv_boxcox(transformed_data, best_lambda)

original_series[:5]

In [ ]:
all(np.isclose(original_series, series.values))

### Dealing with missing values

In [ ]:
# missing data
series_with_nan = series.copy()

idx = np.random.choice(range(len(series)), size=50, replace=False)

series_with_nan[idx] = np.nan

series_with_nan.plot()

In [ ]:
# ffill: propagate last valid observation forward to next valid 
series_with_nan.ffill().plot()
?series_with_nan.ffill

In [ ]:
# bfill: use next valid observation to fill gap.
series_with_nan.bfill().plot()

In [ ]:
# filling nan with mean
series_with_nan.fillna(series.mean()).plot()

### Dealing with irregular series

In [ ]:
# IRREGULAR SERIES
irregular = pd.read_csv('assets/irregular_series.csv')
print(irregular.head())
irregular['Event'] = 'Event'
irregular.set_index('Timestamp', inplace=True)
irregular.index = pd.DatetimeIndex(irregular.index)

irregular.head()

In [ ]:
irregular.index.date

In [ ]:
# grouping by date
regular = irregular.groupby(irregular.index.date).count()

# counting zeros
index_range = pd.date_range('2021-08-14', '2022-04-07')
regular = regular.reindex(index_range, fill_value=0)

regular.reset_index().plot.line(x='index', y='Event', rot=0)

In [ ]:
regular

In [ ]:
regular.resample('W').mean()

In [ ]:
?regular.resample

In [ ]:
regular.resample('M').mean()

### Weekly Seasonality

In [ ]:
from pmdarima.datasets import load_gasoline

?load_gasoline

In [ ]:
gasoline = load_gasoline(True)
gasoline.plot()

In [ ]:
nsdiffs(gasoline, m=52)

In [ ]:
from pmdarima.datasets import load_wineind

?load_wineind